In [1]:
print("print")

print


In [2]:
%pwd

'c:\\Users\\Shiva Kumar\\Desktop\\AI_Medical_ChatBot\\AI_Medical_ChatBot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Shiva Kumar\\Desktop\\AI_Medical_ChatBot\\AI_Medical_ChatBot'

In [5]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\Shiva Kumar\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def load_pdf_files(file_path):
    loader = DirectoryLoader(file_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_docs = load_pdf_files("data")

In [8]:
len(extracted_docs)

637

In [86]:
from typing import List
from langchain_core.documents import Document

In [10]:
def filter_documents(docs:List[Document]) -> List[Document]:
    """Given a list of documents, return a new list of Document objects
    containing only 'source' in metadata and the original page content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        minimal_docs.append(Document(page_content=doc.page_content, metadata={"source": doc.metadata.get("source")}))
    return minimal_docs

In [11]:
minimal_docs = filter_documents(extracted_docs)
# minimal_docs

In [12]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks = text_splitter.split_documents(minimal_docs)
    return chunks

In [13]:
text_chunks = text_split(minimal_docs)
print(f"Number of Chunks :{len(text_chunks)}")

Number of Chunks :5859


In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

def create_embeddings():
    """Create HuggingFace embeddings with device configuration."""
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})
    return embeddings
embeddings = create_embeddings()

C:\Users\Shiva Kumar\AppData\Local\Temp\ipykernel_4640\1437589977.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})


In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
vector = embeddings.embed_query("What is diabetes?")
vector

[-0.026402661576867104,
 0.10922824591398239,
 -0.0924384668469429,
 0.10056494176387787,
 -0.05268426612019539,
 0.02757556363940239,
 0.15084289014339447,
 0.04183783754706383,
 0.005909849423915148,
 0.016620289534330368,
 -0.06484378129243851,
 0.036097265779972076,
 -0.06803156435489655,
 -0.03572404757142067,
 -0.11286158114671707,
 -0.06549476832151413,
 -0.03648458048701286,
 -0.032464418560266495,
 0.022537918761372566,
 0.036317601799964905,
 0.07742998749017715,
 0.094231978058815,
 0.0426589772105217,
 0.017916811630129814,
 0.009462451562285423,
 -0.012717981822788715,
 0.04728841781616211,
 -0.014193284325301647,
 -0.06501471996307373,
 0.006886090151965618,
 -0.02913116104900837,
 0.06316538900136948,
 0.03916837275028229,
 0.048380784690380096,
 -0.08704426884651184,
 0.10905705392360687,
 0.029288042336702347,
 -0.016923056915402412,
 -0.09041677415370941,
 -0.06801263242959976,
 0.035194575786590576,
 -0.03018549643456936,
 -0.005565652158111334,
 0.07018370181322098,

In [17]:
print(len(vector))

384


In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
HUGGINGFACEHUB_API_TOKEN =os.getenv("HUGGINGFACEHUB_API_TOKEN")

if HUGGINGFACEHUB_API_TOKEN is None:
    raise ValueError("HUGGINGFACEHUB_API_TOKEN is not set")

if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY is not set")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [29]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [30]:
pc

In [31]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = "cosine",
        spec = ServerlessSpec(cloud ="aws", region = "us-east-1")
    )
index = pc.Index(index_name)

In [32]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embeddings,
    index_name = index_name
)

In [85]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    embedding = embeddings,
    index_name = index_name
)

In [76]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [77]:
retrieved = retriever.invoke("What is Hypertension?")

In [78]:
retrieved

[Document(id='2d509993-ce81-481c-b12d-d37b018755bb', metadata={'source': 'data\\Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail-\nure or kidney failure. People with high blood pressure\nmay also be at higher risk for heart attacks. Controlling\nhigh blood pressure makes these problems less likely.\nAlpha blockers help lower blood pressure by causing'),
 Document(id='3dee0e61-0f3f-42db-91b1-095ee0aa2c85', metadata={'source': 'data\\Medical_book.pdf'}, page_content='heart and lungs.\n• Seek treatment for hypertension—High blood pressure\ncan be controlled through lifestyle changes—reducing\nsodium and fat, exercising, managing stress, quitting\nKEY TERMS\nArteriosclerosis —Hardening of the arteries. It\nincludes atherosclerosis, but

In [ ]:

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
llm = HuggingFaceEndpoint(
        repo_id='openai/gpt-oss-120b',
        task='text-generation',
        temperature=0.7,
    )
chatModel = ChatHuggingFace(llm=llm)





In [80]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate


In [81]:
system_prompt = (
    "You are a helpful medical assistant. Use the following context to answer the user's question. "
    "If you don't know the answer, just say that you don't know, don't try to make up an answer. "
    "Provide concise and accurate information based on the provided documents."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [82]:
question_answering_chain = create_stuff_documents_chain(chatModel,prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [84]:
response = rag_chain.invoke({"input": "How dengue will get?"})
response['answer']

'Dengue is an **arbovirus** that reaches people through the bite of an infected **mosquito**—most commonly the\u202f* Aedes aegypti*\u202for *Aedes albopictus*\u202fspecies.  \n\n**Transmission steps**\n\n1. **Mosquito acquires the virus** – When a mosquito feeds on the blood of a person (or occasionally an animal) who is already infected with dengue, the virus replicates inside the mosquito.  \n2. **Virus spreads in the mosquito** – The virus multiplies in the mosquito’s salivary glands.  \n3. **Human infection** – During a later blood‑feeding, the infected mosquito injects saliva (and the virus) into a new person’s bloodstream, initiating infection.  \n\nBecause dengue depends on an arthropod vector, the risk peaks during **summer and early fall**, when mosquito populations are highest and the environment is warm and humid. Preventing mosquito bites (e.g., using repellents, screens, eliminating standing water) is the primary way to avoid dengue infection.'